<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>


# Data Formatting (numeric)

This is a process where we make sure we have the right data type.

It is important to realize that while you do formatting you may need some messiness in the values may appear that again again require a cleaning of the column.


# Numeric case

Let's start by reading in some data:

In [ ]:
from IPython.display import IFrame  
wikiLink1="https://en.wikipedia.org/wiki/List_of_ongoing_armed_conflicts#Deaths_by_country"
IFrame(wikiLink1, width=900, height=500)

That table is showing the top 20 countries with deaths per year from 2016 to 2021. Let's bring it:

In [ ]:
import pandas as pd
badCountries = pd.read_html(wikiLink1,flavor='bs4',
                        attrs = {'class': 'wikitable sortable'})
len(badCountries)

The table of interest is here:

In [ ]:
theTable=badCountries[4]
theTable

The first thing to notice is the column names, which is a multi index:

In [ ]:
theTable.columns

What about these new names?

In [ ]:
# concatenate elements

newNames=["_".join((b,a)) for a,b in theTable.columns]
newNames

In [ ]:
# looks better?
betterNames=[n.split('[')[0] for n in newNames]
betterNames

In [ ]:
#Then:
theTable.columns=betterNames
# see
theTable.head()

We do not have multi index anymore. Let's get rid of the first column too: 

In [ ]:
theTable=theTable.iloc[:,1:] # keep all but the first one

Let's check data types:

In [ ]:
theTable.info()

Notice that only two columns that inform the count of deaths have been recognised as numeric. If you leave the table as it is, you will not be able to apply statistical functios to other numeric columns:

In [ ]:
theTable.describe()

Let's explore attention to 2018:

In [ ]:
# where you have non numeric characters:
theTable.Deaths_2018[theTable.Deaths_2018.str.contains(r'\D')]

Check all of them:

In [ ]:
numericColumns=theTable.columns[theTable.columns.str.startswith('Deat')]
numericColumns

In [ ]:
badSymbols=[]
for colname in numericColumns:
    try:
        print(theTable[colname][theTable[colname].str.contains(r'\D')])
    except:
        pass #when numeric columns are found


Let's clean:

In [ ]:
byeStrings=r'\[\d+\]|\*|,|\~|\+'
replaceAllBad=lambda x: x.str.replace(pat=byeStrings,repl="",regex=True)

In [ ]:
#where to apply
theTable[numericColumns].select_dtypes(include='object')

In [ ]:
#then
whereChange=theTable[numericColumns].select_dtypes(include='object').columns
theTable[whereChange]=theTable[whereChange].apply(replaceAllBad)

This should look better:

In [ ]:
theTable

We have a clean version, but:

In [ ]:
theTable.info()

We need to set those columns as numeric:

In [ ]:
toNumeric=lambda x:x.astype('Int64')
theTable[whereChange]=theTable[whereChange].apply(toNumeric)

In [ ]:
theTable.info()

Statistics can be obtained when data is in the rigth type:

In [ ]:
theTable.describe()